In [1]:
import os
import json
import numpy as np
from PIL import Image
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.models as models
from torch.optim import Adam
from tqdm import tqdm

# data_path = "~/scratch/CS_6476_project_code/data/processed_data/"
data_path = "../data/processed_data/"

%load_ext autoreload
%autoreload 2

In [2]:
from utils import create_image_labels_list

seed = 42

# create image labels from annotations
image_labels = create_image_labels_list(data_path)
print(image_labels[0:2])

ModuleNotFoundError: No module named 'utils'

In [3]:
from utils import split_data

# split data
#############################################
# How much train data do we want to use?
train_size = 10000
#############################################
test_size, val_size = 3000, 3000
train_data, val_data, test_data = split_data(
    image_labels, train_size, val_size, test_size, seed
)

num_categories = len(image_labels[0][1])
print(num_categories)

25


In [4]:
print(len(test_data))
print(test_data[0])

print(len(val_data))
print(val_data[0])

print(len(train_data))
print(train_data[0])

# train_data data points vary with train_size, test and val remain constant with everything, as intended

3000
('5fa6c9837bad45c3bbda41c2d9067888.jpg', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0])
3000
('f096684ca8a34f8abc822baa52b36dcf.jpg', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
10000
('451387026390465182797eaecac71544.jpg', [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [5]:
from utils import create_data_loaders

# create Torch DataLoaders
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
image_root_dir = data_path + "images/"
batch_size=32
train_loader, val_loader, test_loader = create_data_loaders(
    train_data, val_data, test_data, image_root_dir, batch_size, transform
)

In [6]:
from models import get_model

In [7]:
model_vgg = get_model("vgg16", tune_conv=True, num_categories=num_categories, dropout_rate=0.5)
print(model_vgg)

/home/hice1/dmishra45/scratch/cv/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/hice1/dmishra45/scratch/cv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
model_resnet = get_model("resnet50", tune_conv=True, num_categories=num_categories, dropout_rate=0.5)
print(model_resnet)

/home/hice1/dmishra45/scratch/cv/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [9]:
for name, param in model_vgg.named_parameters():
    print(f"Layer: {name}, Requires Grad: {param.requires_grad}")

Layer: features.0.weight, Requires Grad: False
Layer: features.0.bias, Requires Grad: False
Layer: features.2.weight, Requires Grad: False
Layer: features.2.bias, Requires Grad: False
Layer: features.5.weight, Requires Grad: False
Layer: features.5.bias, Requires Grad: False
Layer: features.7.weight, Requires Grad: False
Layer: features.7.bias, Requires Grad: False
Layer: features.10.weight, Requires Grad: False
Layer: features.10.bias, Requires Grad: False
Layer: features.12.weight, Requires Grad: False
Layer: features.12.bias, Requires Grad: False
Layer: features.14.weight, Requires Grad: False
Layer: features.14.bias, Requires Grad: False
Layer: features.17.weight, Requires Grad: False
Layer: features.17.bias, Requires Grad: False
Layer: features.19.weight, Requires Grad: False
Layer: features.19.bias, Requires Grad: False
Layer: features.21.weight, Requires Grad: False
Layer: features.21.bias, Requires Grad: False
Layer: features.24.weight, Requires Grad: True
Layer: features.24.bi

In [10]:
for name, param in model_resnet.named_parameters():
    print(f"Layer: {name}, Requires Grad: {param.requires_grad}")

Layer: conv1.weight, Requires Grad: False
Layer: bn1.weight, Requires Grad: False
Layer: bn1.bias, Requires Grad: False
Layer: layer1.0.conv1.weight, Requires Grad: False
Layer: layer1.0.bn1.weight, Requires Grad: False
Layer: layer1.0.bn1.bias, Requires Grad: False
Layer: layer1.0.conv2.weight, Requires Grad: False
Layer: layer1.0.bn2.weight, Requires Grad: False
Layer: layer1.0.bn2.bias, Requires Grad: False
Layer: layer1.0.conv3.weight, Requires Grad: False
Layer: layer1.0.bn3.weight, Requires Grad: False
Layer: layer1.0.bn3.bias, Requires Grad: False
Layer: layer1.0.downsample.0.weight, Requires Grad: False
Layer: layer1.0.downsample.1.weight, Requires Grad: False
Layer: layer1.0.downsample.1.bias, Requires Grad: False
Layer: layer1.1.conv1.weight, Requires Grad: False
Layer: layer1.1.bn1.weight, Requires Grad: False
Layer: layer1.1.bn1.bias, Requires Grad: False
Layer: layer1.1.conv2.weight, Requires Grad: False
Layer: layer1.1.bn2.weight, Requires Grad: False
Layer: layer1.1.bn2.